# Лабораторная работа №1. 
**"Аналитический и численный (Брауна-Робинсон) методы решения антагонистической игры в смешанных стратегиях**

**Выполнил: Александров А. Н., ИУ8-104**

**Вариант: 1**

## Задание
Найти цену игры и оптимальные стратегии обоих игроков методами обратной матрицы (аналитически) и Брауна-Робинсон. Сравните полученные результаты.

Игра Г размера (3 x 3) задана матрицей:
 
| 1  | 11 | 11 |
|:--:|:--:|:--:|
| 7  | 5  | 8  | 
| 16 | 6  | 2  | 


In [1]:
import logging
from copy import deepcopy

import numpy as np

from game_theory.utils.matrix_games.brown_robinson import BrownRobinson
from game_theory.utils.matrix_games.analytical import AnalyticalSolver
from game_theory.utils.matrix_games.game_matrix import GameMatrix
from game_theory.utils.matrix_games.mixed_strategies import (
    check_resulted_game_price,
    get_resulted_mixed_strategies,
)

logging.basicConfig(level=logging.INFO, format='%(message)s')

In [2]:
# Входная матрица прямоугольной игры с нулевой суммой.
original_game_matrix = GameMatrix(
    matrix=np.array(
        [
            [1, 11, 11],
            [7, 5, 8],
            [16, 6, -2],
        ],
        dtype=int,
    )
)
original_game_matrix

+-----------------------------------------------+
|          Таблица стратегий (игрока А)         |
+----------------+----+----+----+---------------+
|   Стратегии    | b1 | b2 | b3 | MIN выигрыш A |
+----------------+----+----+----+---------------+
|       a1       | 1  | 11 | 11 |       1       |
|       a2       | 7  | 5  | 8  |       5       |
|       a3       | 16 | 6  | -2 |       -2      |
| MAX проигрыш B | 16 | 11 | 11 |               |
+----------------+----+----+----+---------------+

In [3]:
print(f"Нижняя цена игры:  {original_game_matrix.lowest_game_price[1]}\n"
      f"Верхняя цена игры: {original_game_matrix.highest_game_price[1]}")

Нижняя цена игры:  5
Верхняя цена игры: 11


In [4]:
game_matrix: GameMatrix = deepcopy(original_game_matrix)
normalizer: int = game_matrix.normalize_matrix()
game_matrix

Прибавили ко всем элементам исходной матрицы 2


+-----------------------------------------------+
|          Таблица стратегий (игрока А)         |
+----------------+----+----+----+---------------+
|   Стратегии    | b1 | b2 | b3 | MIN выигрыш A |
+----------------+----+----+----+---------------+
|       a1       | 3  | 13 | 13 |       3       |
|       a2       | 9  | 7  | 10 |       7       |
|       a3       | 18 | 8  | 0  |       0       |
| MAX проигрыш B | 18 | 13 | 13 |               |
+----------------+----+----+----+---------------+

In [5]:
print(f"Нижняя цена игры:  {game_matrix.lowest_game_price[1]}\n"
      f"Верхняя цена игры: {game_matrix.highest_game_price[1]}")

Нижняя цена игры:  7
Верхняя цена игры: 13


In [6]:
reduced_game: GameMatrix = game_matrix.reduce_dimension(method='dominant_absorption')
reduced_game

+-----------------------------------------------+
|          Таблица стратегий (игрока А)         |
+----------------+----+----+----+---------------+
|   Стратегии    | b1 | b2 | b3 | MIN выигрыш A |
+----------------+----+----+----+---------------+
|       a1       | 3  | 13 | 13 |       3       |
|       a2       | 9  | 7  | 10 |       7       |
|       a3       | 18 | 8  | 0  |       0       |
| MAX проигрыш B | 18 | 13 | 13 |               |
+----------------+----+----+----+---------------+

In [7]:
print(f"Нижняя цена игры:  {reduced_game.lowest_game_price[1]}\n"
      f"Верхняя цена игры: {reduced_game.highest_game_price[1]}")

Нижняя цена игры:  7
Верхняя цена игры: 13


### 1. Аналитический (матричный) метод

#### 1.1. Обратная матрица для игрока A

Для игрока $A$ ($h$ - цена игры; $y_1, ..., y_m$ - смешанные стратегии игрока $A$):
![analytical_A](./img/analytical_A.png)

In [8]:
analytical_solver = AnalyticalSolver(reduced_game)
(
    first_mixed_strategy,
    second_mixed_strategy,
    third_mixed_strategy,
    game_price_value,
) = analytical_solver.player_a_solve()

In [9]:
# Смешанные стратегии игрока A и цена игры.
assert check_resulted_game_price(
    game_matrix=reduced_game,
    game_price_value=game_price_value,
)

mixed_strategies = get_resulted_mixed_strategies(
    player_labels=game_matrix.player_a_strategy_labels,
    labels_to_probability=dict(zip(
        reduced_game.player_a_strategy_labels,
        (first_mixed_strategy, second_mixed_strategy, third_mixed_strategy),
    )),
    player_name="A",
)
print(mixed_strategies)

Цена игры: 7 <= 8.962 <= 13


+------------------------------+
| Смешанные стратегии игрока A |
+---------+---------+----------+
|    a1   |    a2   |    a3    |
+---------+---------+----------+
|  0.295  |  0.513  |  0.192   |
+---------+---------+----------+


In [10]:
sum((first_mixed_strategy, second_mixed_strategy, third_mixed_strategy,))

1.0

#### 2.2.1. Прямая матрица для игрока B
Для игрока $B$ ($g$ - цена игры; $x_1, ..., y_n$ - смешанные стратегии игрока $B$):
![analytical_B](./img/analytical_B.png)

In [11]:
analytical_solver = AnalyticalSolver(reduced_game)
(
    first_mixed_strategy,
    second_mixed_strategy,
    third_mixed_strategy,
    game_price_value,
) = analytical_solver.player_b_solve()

In [12]:
# Смешанные стратегии игрока B и цена игры.
assert check_resulted_game_price(
    game_matrix=reduced_game,
    game_price_value=game_price_value,
)

mixed_strategies = get_resulted_mixed_strategies(
    player_labels=game_matrix.player_b_strategy_labels,
    labels_to_probability=dict(zip(
        reduced_game.player_b_strategy_labels,
        (first_mixed_strategy, second_mixed_strategy, third_mixed_strategy),
    )),
    player_name="B",
)
print(mixed_strategies)

Цена игры: 7 <= 8.962 <= 13


+------------------------------+
| Смешанные стратегии игрока B |
+---------+---------+----------+
|    b1   |    b2   |    b3    |
+---------+---------+----------+
|  0.404  |  0.212  |  0.385   |
+---------+---------+----------+


In [13]:
sum((first_mixed_strategy, second_mixed_strategy, third_mixed_strategy,))

1.0

### Вернёмся к цене игры для исходной матрицы

In [14]:
assert check_resulted_game_price(
    game_matrix=original_game_matrix,
    # Уменьшаем ЦИ на слагаемое, используемое при нормализации матрицы.
    game_price_value=game_price_value - normalizer,
)

Цена игры: 5 <= 6.962 <= 11


### 2. Численный метод Брауна-Робинсон

Уровень погрешности: $\varepsilon \le 0,1$

In [15]:
BrownRobinson(game_matrix=reduced_game).solve()

,Итерация k,Выбор игрока A,Выбор игрока B,x1,x2,x3,y1,y2,y3,Усреднённая оценка ВЦИ,Усреднённая оценка НЦИ,Погрешность ε
